In [2]:

SELECT *
FROM 'onlineretail.csv'
	WHERE CustomerID IS NOT NULL
	AND Quantity > 0
	AND UnitPrice > 0
	AND InvoiceNo NOT LIKE 'C%'
LIMIT 100


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00+00:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00+00:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00+00:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00+00:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00+00:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
95,536378,22352,LUNCH BOX WITH CUTLERY RETROSPOT,6,2010-12-01 09:37:00+00:00,2.55,14688.0,United Kingdom
96,536378,21212,PACK OF 72 RETROSPOT CAKE CASES,120,2010-12-01 09:37:00+00:00,0.42,14688.0,United Kingdom
97,536378,21975,PACK OF 60 DINOSAUR CAKE CASES,24,2010-12-01 09:37:00+00:00,0.55,14688.0,United Kingdom
98,536378,21977,PACK OF 60 PINK PAISLEY CAKE CASES,24,2010-12-01 09:37:00+00:00,0.55,14688.0,United Kingdom


Remove invalid rows

We exclude:

Null CustomerID

Negative quantity (returns)

Zero or negative price

In [1]:
WITH cleaneddata AS (
    SELECT *
    FROM read_csv_auto(
        'onlineretail.csv',
        strict_mode=false,      -- Allow rows with missing columns
        null_padding=true       -- Pad missing columns with NULLs
    )
    WHERE CustomerID IS NOT NULL
      AND Quantity > 0
      AND UnitPrice > 0
)
SELECT *
FROM cleaneddata;

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00+00:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00+00:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00+00:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00+00:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00+00:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
129708,553074,22045,SPACEBOY GIFT WRAP,25,2011-05-13 11:45:00+00:00,0.42,15529.0,United Kingdom
129709,553074,22706,WRAP COWBOYS,25,2011-05-13 11:45:00+00:00,0.42,15529.0,United Kingdom
129710,553074,16161U,WRAP SUKI AND FRIENDS,25,2011-05-13 11:45:00+00:00,0.42,15529.0,United Kingdom
129711,553074,22711,WRAP CIRCUS PARADE,25,2011-05-13 11:45:00+00:00,0.42,15529.0,United Kingdom


Define a reference date

We calculate Recency relative to the last invoice date in the dataset.

In [2]:
SELECT MAX(InvoiceDate) AS max_date
FROM read_csv_auto('onlineretail.csv', strict_mode=false, null_padding=true)

,max_date
0,2011-05-13 11:45:00+00:00


Calculate RFM values per customer
Recency: Days since last purchase

Frequency: Number of unique invoices

Monetary: Total spending

In [3]:
WITH cleaneddata AS (
    SELECT *
    FROM read_csv_auto(
        'onlineretail.csv',
        strict_mode=false,      -- Allow rows with missing columns
        null_padding=true       -- Pad missing columns with NULLs
    )
    WHERE CustomerID IS NOT NULL
      AND Quantity > 0
      AND UnitPrice > 0
),
rfmbase AS (
    SELECT
        CustomerID,
        MAX(InvoiceDate) AS last_purchase_date,
        COUNT(DISTINCT InvoiceNo) AS frequency,
        SUM(Quantity * UnitPrice) AS monetary
    FROM cleaneddata
    GROUP BY CustomerID
)
SELECT
    CustomerID,
    DATE_DIFF('day', last_purchase_date, DATE '2011-05-13') AS recency,
    frequency,
    monetary
FROM rfmbase;

,CustomerID,recency,frequency,monetary
0,13047.0,9,6,1671.68
1,15311.0,3,39,27069.72
2,16098.0,24,3,1010.30
3,17420.0,163,1,130.85
4,16250.0,51,2,389.44
...,...,...,...,...
2571,14853.0,1,1,214.50
2572,14651.0,1,1,416.32
2573,14000.0,1,1,635.66
2574,12757.0,1,2,1160.55


RFM Scoring

We convert raw RFM values into scores from 1–5 using quintiles.

In [5]:
WITH rfm AS (
    SELECT
        CustomerID,
        DATE_DIFF('day', MAX(InvoiceDate), DATE '2011-05-13') AS recency,
        COUNT(DISTINCT InvoiceNo) AS frequency,
        SUM(Quantity * UnitPrice) AS monetary
    FROM read_csv_auto('onlineretail.csv', strict_mode=false, null_padding=true)
    WHERE CustomerID IS NOT NULL
      AND Quantity > 0
      AND UnitPrice > 0
    GROUP BY CustomerID
),
rfmscores AS (
    SELECT
        CustomerID,
        recency,
        frequency,
        monetary,
        NTILE(5) OVER (ORDER BY recency DESC) AS r_score,
        NTILE(5) OVER (ORDER BY frequency) AS f_score,
        NTILE(5) OVER (ORDER BY monetary) AS m_score
    FROM rfm
)
SELECT *
FROM rfmscores;

,CustomerID,recency,frequency,monetary,r_score,f_score,m_score
0,16738.0,87,1,3.75,2,2,1
1,17956.0,39,1,12.75,3,1,1
2,15823.0,162,1,15.00,1,1,1
3,15885.0,99,1,15.00,2,2,1
4,17763.0,53,1,15.00,3,3,1
...,...,...,...,...,...,...,...
2571,14156.0,7,17,42364.23,5,5,5
2572,15749.0,25,3,44534.30,4,4,5
2573,18102.0,23,9,46476.37,4,5,5
2574,12346.0,115,1,77183.60,1,2,5


Customer Segmentation

In [8]:
-- Step 1: Aggregate transactional data at the customer level
WITH customer_agg AS (
    SELECT
        CustomerID,
        MAX(InvoiceDate) AS last_purchase_date,
        COUNT(DISTINCT InvoiceNo) AS frequency,
        SUM(Quantity * UnitPrice) AS monetary
    FROM read_csv_auto('onlineretail.csv', strict_mode=false, null_padding=true)
    WHERE CustomerID IS NOT NULL
      AND Quantity > 0
      AND UnitPrice > 0
    GROUP BY CustomerID
),

-- Step 2: Assign RFM scores using quintiles
rfm_scores AS (
    SELECT
        CustomerID,
        NTILE(5) OVER (
            ORDER BY DATE_DIFF('day', last_purchase_date, DATE '2011-05-13') DESC
        ) AS r_score,
        NTILE(5) OVER (ORDER BY frequency) AS f_score,
        NTILE(5) OVER (ORDER BY monetary) AS m_score
    FROM customer_agg
)

SELECT
    CustomerID,
    r_score,
    f_score,
    m_score,
    CASE
        WHEN r_score >= 4 AND f_score >= 4 AND m_score >= 4 THEN 'Champions'
        WHEN r_score >= 4 AND f_score >= 3 THEN 'Loyal Customers'
        WHEN r_score >= 3 AND m_score >= 3 THEN 'Potential Loyalists'
        WHEN r_score <= 2 AND f_score >= 3 THEN 'At Risk'
        WHEN r_score <= 2 AND m_score <= 2 THEN 'Lost Customers'
        ELSE 'Others'
    END AS customer_segment
FROM rfm_scores;

,CustomerID,r_score,f_score,m_score,customer_segment
0,16738.0,2,2,1,Lost Customers
1,17956.0,3,2,1,Others
2,15823.0,1,1,1,Lost Customers
3,15885.0,2,1,1,Lost Customers
4,17763.0,3,2,1,Others
...,...,...,...,...,...
2571,14156.0,5,5,5,Champions
2572,15749.0,4,4,5,Champions
2573,18102.0,4,5,5,Champions
2574,12346.0,1,1,5,Others
